<a href="https://colab.research.google.com/github/adityakaran/Gerrymandering_AM207/blob/master/Gerrymandering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantifying Gerrymandering in North Carolina

### Collaborators

Aditya Karan, Manish Reddy Vuyyuru, Zizheng Xu

In politics, [Gerrymandering](https://en.wikipedia.org/wiki/Gerrymandering)  is the practice of intentionally manipulating voting districts in a given region to favor a given party. Often this results in relatively outlandish shapes of voting districts. However, the practice remains widespread. In particular, the courts haven't addressed whether one could go to far in explict partisn gerrymandering. The question has often come down to measurabilty - in 2004, Justice Kennedy wrote in his opinion [VIETH V. JUBELIRER](https://www.law.cornell.edu/supct/html/02-1580.ZC.html), states that while the court declained to intervene in this particular map "	the failings of the many proposed standards for measuring the burden a gerrymander imposes on representational rights make our intervention improper.	If workable standards do emerge to measure these burdens, however, courts should be prepared to order relief."

Several measures of partisin gerrymandering have arisen since that time. However, quantifying how much a certian map is biased one way or another alone cannot demosntrate gerrymandering - because of our desire for compactness of districts, keeping relevant natural communites as in the same district and complince of the Voting Rights Act, just scoring a map can't address the problem. 

Luckily there is a solution that can help! If we were able to sample a set of reasonable maps, we can quantify over the possible sets of reasonable graphs - what are the expected numbers of wins etc across this ensamble. Hence Markov Chain Monte Carlo provides a opportunity for us to generate these samples of maps. Once we have a sample we can compute these various statisics and compare to the actual maps to come up with a way of telling whether a given map's statistics would come up by chance or is there a concerted effort to gerrymander. 

We will leverage the GerryChain python package to help organize our data and learn how to start sampling the set of all maps



To start - we've provided data shapefiles/geojson files to aide. These are a best effort means of reconcilling precient level election data and population data - needed to help sample.


In [0]:
'''Add in snipet which reads the file and generate a simple plot of the congressional districts '''

We can already see these shapes are a bit funky.

The two main objects we'll be concerned about are "Graph" objects and "Partition" objects. Graph objects can be geenrate by the Geojson/Shapefiles and give us back a networkx like graph - where the nodes are analogus to precients and the edges represnt two precients being geographically adjacent. 

A partition is a split of the graph into $n$ number of parts. Here we want to partition into $13$ districts. 


In [0]:
'''Show sample graph and partition object'''

Now to run a simple GerryChain example - we can set up a Markov Chain (explain a bit on how to do it and then demo the simpliest chain we can do ). Here, in particular, our "step" in our markov chain is labeled here as "single_flip". Here, we look at the edges in which the nodes are part of two different districts. Out of all those edges, we pick one and then with $\frac{1}{2}$ flip the node to the other district

In [0]:
'''Dummy GerryChain example '''

Congrats - you just run your first chain.!!


.
But something seems odd. Do we really want to choose all districts? 

No of course not! Some districts are "better" than others, and we want to take that in consideration. 

In MCMC language, we consider all possible maps to be sampled from some probablity distribution. We would like to consider the probablity distribution of "reasonable maps" as the probablity distribution we're pulling from. 

To do this  - we'll implement a Metropolis-Hasting criteria and build up our sampling function based on our idea of a "good" map


`

FROM HERE - BREAK DOWN THE SCORING FUNCTION SIMILAR TO WHATS BELOW BUT WITH CODE FOR EACH ONE. Summary below is good  - then breaking each one down

We don't have to show all the nitty gritty code - put most of it in a library and only demo the ones that are relatively interesting. 

The format woudl roughly be text explaining scoring fucntion - implematnion of a part and perhaps a demo of what the score will be 


After that - introduce the idea of weights then the annealing parameter 

After that can be like "let's now try sampling" -> Generate samples with the constraints (can add constraints explanation as well ). 

Plot a few of the samples (can just randomly return some in the chain)

Then also talk about the statistics (efficiency gap/number of wins)

Plot the data of the sample + where the actual district score with some discussion

# Objective

Using resampling tests to determine whether or not a specific districting plans has a suspition of being gerrymandered. The process of resampling from the space of valid redistriction plans is carried out by MCMC methods.

# Approach

Our general procedures of this test include:

(i) sample from all the possibilities of congressional redistricting plans (a.k.a. space of congressional redistricting plans) that adheres to non-partisan redistricting criteria. The way to generate the next sample of redistricting plan given the current one  will be elaborated in **Implementation** section.  

(ii) For each of our sampled redistricting plan, simulate an election result. Aggregate election results to construct the distributions of partisan vote balance on each district and of the congressional delegation’s partisan composition. This distribution is deemed free from any gerrymandering because it is a representive sample of all possible redistrictings.

(iii) check whether the congressional delegation’s partisan composition is commonly seen in the aggregated distribution. When a districting is gerrymandered, it will deviate from the mode of the distribution, end up having extreme values on one of the tails of the distribution.  

# Data 

Voting Tabulation Districts (VTD) geographic data was taken from the NCGA website. [Legislature NC (2010) 2010 VTD geographic data](http://www.ncleg.net/representation/Content/BaseData/BD2016.aspx)







# Implementation


## Mathematics notations

To define the score function, we introduce several mathematical formalisms, the first of which is represents the
state of North Carolina as a graph $ G$ with edges $E$ and vertices $V$ . Each vertex represents a Voting Tabulation District (VTD); an edge between
two vertices exists if the two VTDs share boundaries with non-zero length. In general VTDs may be split into census blocks, however (i)
the utilized redistricting criteria requires this splitting to minimized and (ii) we demonstrate that splitting VTDs to achieve zero population
deviation in a district has nearly no effect on our results.

Defining the graph this way allows us to formally define a redistricting plan: Assuming each VTD belongs to a single district, a redistricting
plan is defined as a function from the vertices, $V$ , to one of the possible districts, which are represented by sequential integers – there are
thirteen congressional districts in North Carolina, so we define a redistricting plan as a function
$\xi : V \rightarrow \{ 1,2 , \dots , 13 \}$.

The redistricting plan
function ξ is interpreted as follows: If a VTD is represented by a vertex $v \in V ,$ then $\xi ( v ) = i$ means that the VTD in question belongs to
district $i$; similarly, for any $i \in \{ 1,2 , \dots , 13 \}$ and plan $\xi ,$ the $i -$ th  district, denoted $D _ { i } ( \xi ) ,$ is given by the set $\{ v \in V : \xi ( v ) = i \}$. 

We restrict
the space of considered redistricting plans $\xi$ such that each district $D _ { i } ( \xi )$ is a single connected component; this restriction, along with our edge criteria, ensures that districts must be contiguous.
We denote the collection of all redistricting plans with connected districts by  $\mathcal { R }$


## MCMC sampling overview

Using a MCMC method to sample possible redistriction plans requires one to generate the next valid redistricting plans given the current one. This requires us to define a probability distribution on the space of redistricting plans. The probability is derived from **score functions**.

## score function
We define this family of distributions based on a score function that evaluates the overall “goodness” of a districting plan. 

The non-partisan design criteria of state law requires every redistricting that:

1. the state population is evenly divided between the thirteen congressional districts,

2. the districts are connected and compact,

3. splitting counties is minimized, and

4. African-American voters are sufficiently concentrated in two districts to affect the winner.

The first three criteria come from House Bill 92 (HB92) of the NC General Assembly


A plan ξ is rated with our score function denoted J. J maps each redistricting ξ ∈ R to a nonnegative number. Lower scores signify redistricting plans that more closely adhere to the criteria of HB92. We break down the score function J into four components that
measure how well a given redistricting satisfies the individual principles outlined in HB92:

1. the population score $J _ { p } ( \xi )$ : measures how well the redistricting ξ partitions the population of North Carolina into 13 equal population
groups; 

2. the isoperimetric score  $J _ { I } ( \xi ) $: measures how compact the districts are; 

3. the county score $J _ { c } ( \xi )$: measures the number of counties split between multiple districts; 

4. the minority score  $J _ { m } ( \xi )$ measures the extent to which a districting plan adheres to the VRA

Then our score function $J$ is defined as a weighted sum of $J _ { p } , J _ { I } , J _ { c } ,$ and $J _ { m }$

$J ( \xi ) = w _ { p } J _ { p } ( \xi ) + w _ { I } J _ { I } ( \xi ) + w _ { c } J _ { c } ( \xi ) + w _ { m } J _ { m } ( \xi )$

where $w _ { p } , w _ { I } , w _ { c } ,$ and $w _ { m }$ are a collection of positive weights determined by an iterative process.


### The population score function  $J _ { p } ( \xi )$
The population score, which measures how evenly populated the districts are, is defined by

$$J _ { p } ( \xi ) = \sqrt { \sum _ { i = 1 } ^ { 13 } \left( \frac { \operatorname { pop } \left( D _ { i } ( \xi ) \right) } { \text { pop}_{\text {ldeal}}  } - 1 \right) ^ { 2 } }, \ \mathrm { pop } _ { \text {ldeal} } = \frac { N _ { \text {pop} } } { 13 }$$

where $N _ {$pop$}$ is the total population of North Carolina, pop$\left( D _ { i } ( \xi ) \right)$ is the population of the district $D _ { i } ( \xi )$.


### The Isoparametric score function  $J _ { I } ( \xi ) $

The Isoperimetric score, which measures the overall compactness of a redistricting, is defined by

$$J _ { I } ( \xi ) = \sum _ { i = 1 } ^ { 13 } \frac { \left[ \mathrm { boundary } \left( \partial D _ { i } ( \xi ) \right) \right] ^ { 2 } } { \operatorname { area } \left( D _ { i } ( \xi ) \right) }$$

It is is the ratio of the square perimeter to the total area of each district. The Isoparametric score is minimized for a circle, which is the most
compact shape.

### The county score function $J _ { c } ( \xi )$

### The Voting Rights Act or minority score function $J _ { m } ( \xi )$

## probability distribution
Then use the score function to
define a **probability distribution**. 


The acceptance probability is
given by:
